# Coffee Price Asymmetry Analysis

This notebook examines asymmetric price transmission ("rockets and feathers") in the U.S. coffee market,
analyzing how retail ground coffee prices respond to changes in wholesale coffee futures prices.

**Methodology:**
- Distributed lag model with HAC (Newey-West) standard errors
- Symmetric and asymmetric specifications
- Robustness checks: lag selection, outlier trimming, placebo leads

**Data:**
- Retail prices: Nielsen scanner data (weekly, 2013–2023)
- Upstream costs: Coffee futures (ICE Coffee C)

In [1]:

# =============================================================================
# Imports and Configuration
# =============================================================================

from __future__ import annotations

import os
from pathlib import Path
from typing import TYPE_CHECKING

import duckdb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

if TYPE_CHECKING:
    from statsmodels.regression.linear_model import RegressionResultsWrapper

# =============================================================================
# Configuration
# =============================================================================

# Berkeley brand colors
BERKELEY_BLUE = "#002676"
CALIFORNIA_GOLD = "#FDB515"
DARK_GREEN = "#00553A"
DARK_PURPLE = "#431170"
MEDIUM_ROSE = "#E7115E"

# Paths - adjust these for your environment
DATA_FOLDER = Path("/home/akimovh/rockets_feathers/coffee/data")
PLOT_FOLDER = Path("/home/akimovh/rockets_feathers/coffee/plots")

# Database connection
con = duckdb.connect(str(DATA_FOLDER / "coffee_nielsen" / "coffee_nielsen.duckdb"))
con.execute(f"PRAGMA threads={os.cpu_count()}")

## 1. Data Preparation

Merge weekly retail prices from Nielsen scanner data with coffee futures prices.

In [2]:
# =============================================================================
# Load and Process Futures (Wholesale Cost) Data
# =============================================================================

futures_path = DATA_FOLDER / "invest" / "coffee_futures.csv"
futures_df = pd.read_csv(futures_path, usecols=["Date", "Price"])
futures_df.columns = ["date", "up_price"]
futures_df["date"] = pd.to_datetime(futures_df["date"])
futures_df = futures_df.sort_values("date")


# Now set index for resampling
futures_df = futures_df.set_index("date")

# Resample to weekly frequency (Saturday end) and convert to cents per ounce
# Coffee futures are quoted in cents per pound; divide by 16 for cents per ounce
cost_df = (
    futures_df["up_price"]
    .resample("W-SAT", label="right", closed="right")
    .mean()
    .reset_index()
    .rename(columns={"up_price": "cost"})
)

# Convert to cents per ounce
cost_df["cost"] = cost_df["cost"] / 16

# Apply 13-week centered moving average (approximately 1 quarter)
# cost_df["cost"] = cost_df["cost"].rolling(window=13, center=True).mean()

# Drop rows with NaN from the centered MA
cost_df = cost_df.dropna().copy()

print(f"Cost data: {len(cost_df)} observations from {cost_df['date'].min()} to {cost_df['date'].max()}")

# =============================================================================
# Load Retail Price Data from Nielsen
# =============================================================================

price_df = con.execute(
    """
    SELECT week_end AS date,
           avg_price * 100 AS price
    FROM GROUND_COFFEE_PRICE_AGG
    """
).df()

# =============================================================================
# Merge Datasets
# =============================================================================

df_weekly = pd.merge(price_df, cost_df, on="date", how="inner")

print(f"Sample period: {df_weekly['date'].min()} to {df_weekly['date'].max()}")
print(f"Observations: {len(df_weekly)}")
df_weekly['lp'] = np.log(df_weekly['price'])
df_weekly['lc'] = np.log(df_weekly['cost'])
df_weekly.head()

Cost data: 735 observations from 2012-01-07 00:00:00 to 2026-01-31 00:00:00
Sample period: 2013-01-05 00:00:00 to 2023-12-30 00:00:00
Observations: 574


,date,price,cost,lp,lc
0,2013-01-05,41.438943,9.172656,3.724221,2.216227
1,2013-01-12,39.702247,9.368125,3.681408,2.237313
2,2013-01-19,40.104088,9.632500,3.691478,2.265143
3,2013-01-26,39.584277,9.278906,3.678432,2.227744
4,2013-02-02,40.349617,9.267500,3.697582,2.226514


## 2. Visualization

Plot retail ground coffee prices against coffee futures prices over time.

In [3]:
# =============================================================================
# Time Series Plot: Retail vs. Futures Prices (Side by Side)
# =============================================================================

from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Retail Price vs. Coffee Futures Price", 
                    "Log Retail Price vs. Log Coffee Futures Price"),
    horizontal_spacing=0.12
)

# Left plot: Original prices
fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["price"],
        mode="lines",
        name="Retail",
        line=dict(color=BERKELEY_BLUE, width=3),
        showlegend=True,
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["cost"],
        mode="lines",
        name="Futures",
        line=dict(color=CALIFORNIA_GOLD, width=3),
        showlegend=True,
    ),
    row=1, col=1
)

# Right plot: Log prices
fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["lp"],
        mode="lines",
        name="Retail",
        line=dict(color=BERKELEY_BLUE, width=3),
        showlegend=False,
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=df_weekly["date"],
        y=df_weekly["lc"],
        mode="lines",
        name="Futures",
        line=dict(color=CALIFORNIA_GOLD, width=3),
        showlegend=False,
    ),
    row=1, col=2
)

fig.update_xaxes(
    title_text="Date",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=1
)

fig.update_xaxes(
    title_text="Date",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=2
)

fig.update_yaxes(
    title_text="Price (Cents per Ounce)",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=1
)

fig.update_yaxes(
    title_text="Log Price",
    showgrid=True, gridcolor="lightgray", zeroline=False,
    linecolor="black", linewidth=1, mirror=True,
    ticks="outside", tickwidth=1, tickcolor="black",
    row=1, col=2
)

fig.update_layout(
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=150),
    width=1200,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / "Price_Cost.png")

## 3. Symmetric Pass-Through Model

Estimate the distributed lag model:

$$\Delta \ln(p_t) = \alpha + \sum_{\ell=0}^{K} \beta_\ell \Delta \ln(c_{t-\ell}) + \gamma_m + \varepsilon_t$$

where $p_t$ is retail price, $c_t$ is wholesale cost, and $\gamma_m$ are month fixed effects.

In [4]:
def estimate_passthrough(
    df_weekly: pd.DataFrame,
    K_c: int = 52,
    K_p: int = 4,
    L_leads: int = 0,
    outlier_winsorize: bool = False,
    hac_lags: int = 26,
) -> tuple[RegressionResultsWrapper, list[str], list[str]]:
    """
    Estimate symmetric distributed lag pass-through model with price inertia.
    
    Estimates the model:
        Δln(p_t) = α + Σ β_k Δln(c_{t-k}) + Σ γ_j Δln(p_{t-j}) + γ_m + ε_t
    
    where p_t is retail price, c_t is wholesale cost, and γ_m are month fixed effects.
    
    Parameters
    ----------
    df_weekly : pd.DataFrame
        Weekly data with columns: date, price, cost, lp (log price), lc (log cost)
    K_c : int, default=52
        Number of cost lags (weeks)
    K_p : int, default=4
        Number of price lags for autoregressive component
    L_leads : int, default=0
        Number of leads for placebo test
    outlier_winsorize : bool, default=False
        Whether to winsorize cost changes at 1st/99th percentiles
    hac_lags : int, default=26
        Number of lags for HAC standard errors (Newey-West)
    
    Returns
    -------
    model : RegressionResultsWrapper
        Fitted OLS model with HAC standard errors
    lead_vars : list[str]
        Names of lead variables (empty if L_leads=0)
    lag_vars : list[str]
        Names of all lag variables (cost + price lags)
    """
    df = df_weekly.copy()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)

    # Log transformations and first differences
    df["d_lp"] = df["lp"].diff()
    df["d_lc"] = df["lc"].diff()

    # Winsorize outliers if requested
    if outlier_winsorize:
        p01, p99 = df["d_lc"].quantile([0.01, 0.99])
        n_capped = ((df["d_lc"] < p01) | (df["d_lc"] > p99)).sum()
        df["d_lc"] = df["d_lc"].clip(lower=p01, upper=p99)
        print(f"Winsorization: capped {n_capped} observations at 1st/99th percentiles")

    # Create ALL lag variables at once to avoid fragmentation
    lag_dict = {}
    
    # Cost lags
    for lag in range(K_c + 1):
        lag_dict[f"d_lc_l{lag}"] = df["d_lc"].shift(lag)
    
    # Price lags
    for lag in range(1, K_p + 1):
        lag_dict[f"d_lp_l{lag}"] = df["d_lp"].shift(lag)
    
    # Leads (placebo)
    for lead in range(1, L_leads + 1):
        lag_dict[f"d_lc_lead{lead}"] = df["d_lc"].shift(-lead)
    
    # Create DataFrame from dict and concatenate at once
    lag_df = pd.DataFrame(lag_dict, index=df.index)
    df = pd.concat([df, lag_df], axis=1)

    # Month fixed effects
    df["month"] = df["date"].dt.month
    month_dummies = pd.get_dummies(df["month"], prefix="m", drop_first=True, dtype=int)
    df = pd.concat([df, month_dummies], axis=1)

    # Drop NAs and estimate
    df = df.dropna().copy()

    # Build regressor lists
    lag_vars_cost = [f"d_lc_l{lag}" for lag in range(K_c + 1)]
    lag_vars_price = [f"d_lp_l{lag}" for lag in range(1, K_p + 1)]
    lag_vars = lag_vars_cost + lag_vars_price
    lead_vars = [f"d_lc_lead{lead}" for lead in range(1, L_leads + 1)]
    
    X_cols = lead_vars + lag_vars + list(month_dummies.columns)

    y = df["d_lp"]
    X = sm.add_constant(df[X_cols])

    model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})

    return model, lead_vars, lag_vars


def compute_cumulative_pt(
    model: RegressionResultsWrapper,
    K_c: int,
    K_p: int,
) -> np.ndarray:
    """
    Compute cumulative pass-through for symmetric model with price inertia.
    
    Parameters
    ----------
    model : RegressionResultsWrapper
        Fitted symmetric model
    K_c : int
        Number of cost lags
    K_p : int
        Number of price lags
    
    Returns
    -------
    np.ndarray
        Cumulative pass-through path of length K_c + 1
    """
    params = model.params
    
    # Extract coefficients
    beta = {i: params.get(f'd_lc_l{i}', 0) for i in range(K_c + 1)}
    gamma = {i: params.get(f'd_lp_l{i}', 0) for i in range(1, K_p + 1)}
    
    # Compute cumulative pass-through
    B = [beta[0]]
    
    for k in range(1, K_c + 1):
        # Direct effect from cost change at lag k
        term_beta = beta.get(k, 0)
        
        # Autoregressive feedback from previous price changes
        term_ar = 0
        for j in range(1, min(K_p + 1, k + 1)):
            if k - j >= 0:
                if k - j == 0:
                    prev_change = B[0]
                else:
                    prev_change = B[k - j] - B[k - j - 1]
                term_ar += gamma.get(j, 0) * prev_change
        
        # Total change in pass-through at period k
        delta_B_k = term_beta + term_ar
        B.append(B[k - 1] + delta_B_k)
    
    return np.array(B)

In [5]:
# =============================================================================
# Baseline Estimation
# =============================================================================

K_c = 52  # Number of lags
K_p = 4
L = 0   # Number of leads (placebo)

model, lead_vars, lag_vars = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, L_leads = L)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   d_lp   R-squared:                       0.330
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     31.50
Date:                Sun, 01 Feb 2026   Prob (F-statistic):          3.45e-133
Time:                        18:00:14   Log-Likelihood:                 1184.7
No. Observations:                 521   AIC:                            -2231.
Df Residuals:                     452   BIC:                            -1938.
Df Model:                          68                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0070      0.005      1.408      0.1

In [6]:
# -----------------------------
# 1) Settings
# -----------------------------
H = 60          # horizon to compute IRF/cumulative PT (should be > K_c due to inertia)
N_DRAWS = 3000   # parametric draws for CI (2000–5000 is usually fine)
SEED = 42

# -----------------------------
# 2) Helper: compute cum PT from a parameter vector
#    Model: d_lp_t = Σ β_k d_lc_{t-k} + Σ γ_j d_lp_{t-j} + ...
#    Shock: d_lc_t = 1, others 0
# -----------------------------
def cum_pt_from_params(beta, gamma, K_c, K_p, H):
    """
    beta: array length K_c+1 for d_lc_l0..d_lc_lKc
    gamma: array length K_p (for d_lp_l1..d_lp_lKp)
    returns: cumulative_pt (length H+1), irf_dp (Δp response, length H+1)
    """
    irf_dp = np.zeros(H + 1)
    irf_dp[0] = beta[0]

    for h in range(1, H + 1):
        bh = beta[h] if h <= K_c else 0.0
        ar = 0.0
        for j in range(1, min(K_p, h) + 1):
            ar += gamma[j - 1] * irf_dp[h - j]
        irf_dp[h] = bh + ar

    cumulative_pt = irf_dp.cumsum()
    return cumulative_pt, irf_dp


# -----------------------------
# 3) Point estimate curve
# -----------------------------
names = model.model.exog_names
idx = {n: i for i, n in enumerate(names)}
theta_hat = model.params.values

# Pull β and γ from the fitted model (point estimates)
beta_hat = np.array([model.params.get(f"d_lc_l{k}", 0.0) for k in range(K_c + 1)])
gamma_hat = np.array([model.params.get(f"d_lp_l{j}", 0.0) for j in range(1, K_p + 1)])

cumulative_pt, irf_dp = cum_pt_from_params(beta_hat, gamma_hat, K_c, K_p, H)


# -----------------------------
# 4) Correct 95% CI via parametric simulation
#    Draw θ ~ N(θhat, Vhat) using full cov matrix from the regression,
#    recompute curve each draw, take percentiles at each horizon.
# -----------------------------
V = model.cov_params().values
rng = np.random.default_rng(SEED)
draws = rng.multivariate_normal(theta_hat, V, size=N_DRAWS)

# Precompute indexes for the parameters we need in each draw
beta_idx = [idx[f"d_lc_l{k}"] for k in range(K_c + 1)]
gamma_idx = [idx[f"d_lp_l{j}"] for j in range(1, K_p + 1)]

cum_draws = np.zeros((N_DRAWS, H + 1))

for r in range(N_DRAWS):
    th = draws[r]
    beta_r = th[beta_idx]
    gamma_r = th[gamma_idx]
    cum_r, _ = cum_pt_from_params(beta_r, gamma_r, K_c, K_p, H)
    cum_draws[r] = cum_r

ci_lower = np.percentile(cum_draws, 2.5, axis=0)
ci_upper = np.percentile(cum_draws, 97.5, axis=0)

horizons = np.arange(H + 1)


# -----------------------------
# 5) Plot
# -----------------------------
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.concatenate([horizons, horizons[::-1]]),
        y=np.concatenate([ci_upper, ci_lower[::-1]]),
        fill="toself",
        fillcolor="rgba(0, 38, 118, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        showlegend=True,
        name="95% CI",
        hoverinfo="skip",
    )
)

fig.add_trace(
    go.Scatter(
        x=horizons,
        y=cumulative_pt,
        mode="lines+markers",
        name="Cumulative pass-through",
        line=dict(color=BERKELEY_BLUE, width=3),
        marker=dict(size=6, color=BERKELEY_BLUE),
    )
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1.5)
fig.add_hline(y=1, line_dash="dot", line_color="red", line_width=1.5)

fig.update_layout(
    title="Cumulative Pass-Through with Price Inertia",
    xaxis_title="Weeks after cost shock (h)",
    yaxis_title="Cumulative pass-through",
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    xaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=150),
    width=850,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / "Cumulative_PassThrough.png")


# -----------------------------
# 6) Print a few horizons (more meaningful than K_p)
# -----------------------------
for h in [4, 13, 26, 52, 104, 200]:
    if h <= H:
        print(f"Pass-through at week {h}: {cumulative_pt[h]:.3f} "
              f"[{ci_lower[h]:.3f}, {ci_upper[h]:.3f}]")


Pass-through at week 4: 0.033 [-0.031, 0.098]
Pass-through at week 13: -0.018 [-0.089, 0.049]
Pass-through at week 26: 0.053 [-0.027, 0.128]
Pass-through at week 52: 0.143 [0.028, 0.264]


## 4. Diagnostics

In [7]:
# =============================================================================
# Robustness Check 1: Placebo Leads (Timing Check)
# =============================================================================

K_c = 52  # Number of lags
K_p = 4
L = 4   # Number of leads (placebo)

print("=" * 70)
print(f"PLACEBO TEST: Adding L={L} leads to baseline model (K_c={K_c}, K_p={K_p})")
print("=" * 70)

m_placebo, lead_vars, lag_vars = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, L_leads = L)

# Joint test on leads
hypotheses = f"{', '.join(lead_vars)} = 0"
f_test = m_placebo.f_test(hypotheses)

print(f"\nJoint test H₀: all lead coefficients = 0")
print(f"F-statistic: {f_test.fvalue:.4f}")
print(f"p-value:     {f_test.pvalue:.6f}")
print(f"df:          ({f_test.df_num}, {f_test.df_denom})")

if f_test.pvalue < 0.05:
    print("\n⚠️  WARNING: Placebo leads are jointly significant (p < 0.05)!")
else:
    print("\n✓ Placebo leads are NOT significant (timing check passed)")

# Individual lead coefficients
print("\n" + "-" * 70)
print("Individual lead coefficients:")
print("-" * 70)

lead_results = pd.DataFrame({
    "Variable": lead_vars,
    "Coef": [f"{m_placebo.params[v]:.4f}" for v in lead_vars],
    "Std.Err": [f"{m_placebo.bse[v]:.4f}" for v in lead_vars],
    "t": [f"{m_placebo.tvalues[v]:.2f}" for v in lead_vars],
    "p-value": [f"{m_placebo.pvalues[v]:.4f}" for v in lead_vars],
})
print(lead_results.to_string(index=False))

PLACEBO TEST: Adding L=4 leads to baseline model (K_c=52, K_p=4)

Joint test H₀: all lead coefficients = 0
F-statistic: 0.7595
p-value:     0.552053
df:          (4.0, 444.0)

✓ Placebo leads are NOT significant (timing check passed)

----------------------------------------------------------------------
Individual lead coefficients:
----------------------------------------------------------------------
  Variable    Coef Std.Err     t p-value
d_lc_lead1 -0.0300  0.0254 -1.18  0.2371
d_lc_lead2 -0.0367  0.0325 -1.13  0.2577
d_lc_lead3  0.0330  0.0320  1.03  0.3021
d_lc_lead4 -0.0030  0.0418 -0.07  0.9431


In [8]:
# =============================================================================
# Robustness Check 2: Lag Selection
# =============================================================================

K_p = 4
L = 0  

K_values = [13, 26, 39, 52, 104]
horizons_to_report = [13, 26, 39, 52, 104]

# Using compute_cumulative_pt defined earlier
results = {}
for K in K_values:
    m, _, _ = estimate_passthrough(df_weekly, K_c=K, K_p=K_p, L_leads=L)
    results[K] = {"cumulative_pt": compute_cumulative_pt(m, K, K_p)}

# Summary table
table_data = []
for K in K_values:
    row = {"K": K}
    for h in horizons_to_report:
        if h <= K:
            row[f"h={h}"] = f"{results[K]['cumulative_pt'][h]:.4f}"
        else:
            row[f"h={h}"] = "—"
    table_data.append(row)

pd.DataFrame(table_data)

In [9]:
# =============================================================================
# Robustness Check 3: Outlier Sensitivity
# =============================================================================

K_c = 52
K_p = 4

# Using compute_cumulative_pt defined earlier
m_baseline, _, _ = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, outlier_winsorize=False)
m_trimmed, _, _ = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, outlier_winsorize=True)

beta_baseline = compute_cumulative_pt(m_baseline, K_c, K_p)
beta_trimmed = compute_cumulative_pt(m_trimmed, K_c, K_p)

print(f"Cumulative PT at h=13: Baseline={beta_baseline[13]:.4f}, Trimmed={beta_trimmed[13]:.4f}")
print(f"Cumulative PT at h=26: Baseline={beta_baseline[26]:.4f}, Trimmed={beta_trimmed[26]:.4f}")
print(f"Cumulative PT at h=52: Baseline={beta_baseline[52]:.4f}, Trimmed={beta_trimmed[52]:.4f}")

In [10]:
# =============================================================================
# Robustness Check 4: HAC Lag Sensitivity
# =============================================================================

K_c = 52
K_p = 4
hac_values = [8, 26, 52]
horizons_to_check = [8, 26, 52]

print("=" * 80)
print("HAC LAG SENSITIVITY CHECK")
print("=" * 80)

# Using compute_cumulative_pt defined earlier
# Function to compute bootstrap CI
def compute_bootstrap_ci(model, K_c, K_p, n_boot=500):
    cost_vars = [f'd_lc_l{i}' for i in range(K_c + 1)]
    price_vars = [f'd_lp_l{i}' for i in range(1, K_p + 1)]
    all_vars = cost_vars + price_vars
    
    param_vec = model.params[all_vars].values
    vcov_matrix = model.cov_params().loc[all_vars, all_vars].values
    
    np.random.seed(42)
    bootstrap_samples = np.random.multivariate_normal(param_vec, vcov_matrix, n_boot)
    
    bootstrap_curves = []
    for i in range(n_boot):
        boot_params = bootstrap_samples[i]
        beta_boot = {j: boot_params[j] for j in range(K_c + 1)}
        gamma_boot = {j: boot_params[K_c + j] for j in range(1, K_p + 1)}
        
        # Compute cumulative PT for this bootstrap sample
        B_boot = [beta_boot[0]]
        for k in range(1, K_c + 1):
            term_beta = beta_boot.get(k, 0)
            term_ar = 0
            for j in range(1, min(K_p + 1, k + 1)):
                if k - j >= 0:
                    prev_change = B_boot[0] if k - j == 0 else B_boot[k-j] - B_boot[k-j-1]
                    term_ar += gamma_boot.get(j, 0) * prev_change
            delta_B_k = term_beta + term_ar
            B_boot.append(B_boot[k-1] + delta_B_k)
        
        bootstrap_curves.append(B_boot)
    
    bootstrap_curves = np.array(bootstrap_curves)
    ci_lower = np.percentile(bootstrap_curves, 2.5, axis=0)
    ci_upper = np.percentile(bootstrap_curves, 97.5, axis=0)
    
    return ci_lower, ci_upper

results_hac = {}
for hac in hac_values:
    m, _, _ = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, hac_lags=hac)
    cumulative_pt = compute_cumulative_pt(m, K_c, K_p)
    ci_lower, ci_upper = compute_bootstrap_ci(m, K_c, K_p, n_boot=500)
    
    # Joint F-test on all cost lags
    cost_vars = [f'd_lc_l{i}' for i in range(K_c + 1)]
    hypotheses = f"{', '.join(cost_vars)} = 0"
    f_test = m.f_test(hypotheses)
    
    results_hac[hac] = {
        "cumulative_pt": cumulative_pt,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper,
        "f_stat": f_test.fvalue,
        "f_pval": f_test.pvalue,
    }

# Summary table
print("\nCumulative Pass-Through at Key Horizons:")
for hac in hac_values:
    print(f"\nHAC lags = {hac}:")
    for h in horizons_to_check:
        pt = results_hac[hac]["cumulative_pt"][h]
        ci_l = results_hac[hac]["ci_lower"][h]
        ci_u = results_hac[hac]["ci_upper"][h]
        se = (ci_u - ci_l) / (2 * 1.96)
        print(f"  h={h}: {pt:.4f} (SE≈{se:.4f}), 95% CI: [{ci_l:.4f}, {ci_u:.4f}]")
    
    print(f"\nJoint F-test (all cost lags = 0): F={results_hac[hac]['f_stat']:.2f}, p={results_hac[hac]['f_pval']:.4f}")

In [11]:
# =============================================================================
# Robustness Check 2: Price Lag Selection
# =============================================================================

K_c = 52
L = 0  

K_p_values = [0, 2, 4, 8]
horizons_to_report = [13, 26, 39, 52]

# Using compute_cumulative_pt defined earlier
results = {}
for K_p in K_p_values:
    m, _, _ = estimate_passthrough(df_weekly, K_c=K_c, K_p=K_p, L_leads=L)
    results[K_p] = {"cumulative_pt": compute_cumulative_pt(m, K_c, K_p)}

# Summary table
table_data = []
for K_p in K_p_values:
    row = {"K_p": K_p}
    for h in horizons_to_report:
        row[f"h={h}"] = f"{results[K_p]['cumulative_pt'][h]:.4f}"
    table_data.append(row)

pd.DataFrame(table_data)

# 5. Asymmetric Pass-Through model

In [12]:
def estimate_passthrough_asymmetric(
    df_weekly: pd.DataFrame,
    K_c: int = 52,
    K_p: int = 4,
    L_leads: int = 0,
    outlier_winsorize: bool = False,
    hac_lags: int = 26,
) -> tuple[RegressionResultsWrapper, list[str], list[str], list[str], list[str], list[str]]:
    """
    Estimate asymmetric distributed lag pass-through model with price inertia.
    
    Splits cost changes and price inertia into positive and negative components
    to test for asymmetric price transmission ("rockets and feathers").
    
    Parameters
    ----------
    df_weekly : pd.DataFrame
        Weekly data with columns: date, price, cost, lp (log price), lc (log cost)
    K_c : int, default=52
        Number of cost lags (weeks)
    K_p : int, default=4
        Number of price lags for autoregressive component
    L_leads : int, default=0
        Number of leads for placebo test
    outlier_winsorize : bool, default=False
        Whether to winsorize cost changes at 1st/99th percentiles
    hac_lags : int, default=26
        Number of lags for HAC standard errors
    
    Returns
    -------
    model : RegressionResultsWrapper
        Fitted OLS model with HAC standard errors
    lead_vars : list[str]
        Names of lead variables (empty if L_leads=0)
    lag_vars_cost_pos : list[str]
        Names of positive cost lag variables
    lag_vars_cost_neg : list[str]
        Names of negative cost lag variables
    lag_vars_price_pos : list[str]
        Names of positive price lag variables
    lag_vars_price_neg : list[str]
        Names of negative price lag variables
    """
    df = df_weekly.copy()
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)

    # Log transformations and first differences
    df["d_lp"] = df["lp"].diff()
    df["d_lc"] = df["lc"].diff()

    # Winsorize outliers if requested
    if outlier_winsorize:
        p01, p99 = df["d_lc"].quantile([0.01, 0.99])
        n_capped = ((df["d_lc"] < p01) | (df["d_lc"] > p99)).sum()
        df["d_lc"] = df["d_lc"].clip(lower=p01, upper=p99)
        print(f"Winsorization: capped {n_capped} observations at 1st/99th percentiles")

    # Split cost changes into positive and negative
    df["d_lc_pos"] = df["d_lc"].clip(lower=0)
    df["d_lc_neg"] = df["d_lc"].clip(upper=0)
    
    # Split price changes into positive and negative
    df["d_lp_pos"] = df["d_lp"].clip(lower=0)
    df["d_lp_neg"] = df["d_lp"].clip(upper=0)

    # Create ALL lag variables at once to avoid fragmentation
    lag_dict = {}
    
    # Cost lags - POSITIVE
    for lag in range(K_c + 1):
        lag_dict[f"d_lc_pos_l{lag}"] = df["d_lc_pos"].shift(lag)
    
    # Cost lags - NEGATIVE
    for lag in range(K_c + 1):
        lag_dict[f"d_lc_neg_l{lag}"] = df["d_lc_neg"].shift(lag)
    
    # Price lags - POSITIVE
    for lag in range(1, K_p + 1):
        lag_dict[f"d_lp_pos_l{lag}"] = df["d_lp_pos"].shift(lag)
    
    # Price lags - NEGATIVE
    for lag in range(1, K_p + 1):
        lag_dict[f"d_lp_neg_l{lag}"] = df["d_lp_neg"].shift(lag)
    
    # Leads (placebo) - split into positive and negative
    for lead in range(1, L_leads + 1):
        lag_dict[f"d_lc_pos_lead{lead}"] = df["d_lc_pos"].shift(-lead)
        lag_dict[f"d_lc_neg_lead{lead}"] = df["d_lc_neg"].shift(-lead)
    
    # Create DataFrame from dict and concatenate at once
    lag_df = pd.DataFrame(lag_dict, index=df.index)
    df = pd.concat([df, lag_df], axis=1)

    # Month fixed effects
    df["month"] = df["date"].dt.month
    month_dummies = pd.get_dummies(df["month"], prefix="m", drop_first=True, dtype=int)
    df = pd.concat([df, month_dummies], axis=1)

    # Drop NAs and estimate
    df = df.dropna().copy()

    # Build regressor lists
    lag_vars_cost_pos = [f"d_lc_pos_l{lag}" for lag in range(K_c + 1)]
    lag_vars_cost_neg = [f"d_lc_neg_l{lag}" for lag in range(K_c + 1)]
    lag_vars_price_pos = [f"d_lp_pos_l{lag}" for lag in range(1, K_p + 1)]
    lag_vars_price_neg = [f"d_lp_neg_l{lag}" for lag in range(1, K_p + 1)]
    
    lag_vars = lag_vars_cost_pos + lag_vars_cost_neg + lag_vars_price_pos + lag_vars_price_neg
    
    lead_vars_pos = [f"d_lc_pos_lead{lead}" for lead in range(1, L_leads + 1)]
    lead_vars_neg = [f"d_lc_neg_lead{lead}" for lead in range(1, L_leads + 1)]
    lead_vars = lead_vars_pos + lead_vars_neg
    
    X_cols = lead_vars + lag_vars + list(month_dummies.columns)

    y = df["d_lp"]
    X = sm.add_constant(df[X_cols])

    model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": hac_lags})

    return model, lead_vars, lag_vars_cost_pos, lag_vars_cost_neg, lag_vars_price_pos, lag_vars_price_neg


def compute_cumulative_pt_asymmetric(
    model: RegressionResultsWrapper,
    K_c: int,
    K_p: int,
    shock_type: str = "positive",
) -> np.ndarray:
    """
    Compute cumulative pass-through for asymmetric model.
    
    Parameters
    ----------
    model : RegressionResultsWrapper
        Fitted asymmetric model (or MockModel with params.get interface)
    K_c : int
        Number of cost lags
    K_p : int
        Number of price lags
    shock_type : str, default="positive"
        Either "positive" or "negative" to specify which shock to trace
    
    Returns
    -------
    np.ndarray
        Cumulative pass-through path of length K_c + 1
    """
    params = model.params
    
    # Extract coefficients based on shock type
    if shock_type == "positive":
        beta = {i: params.get(f'd_lc_pos_l{i}', 0) for i in range(K_c + 1)}
        gamma_pos = {i: params.get(f'd_lp_pos_l{i}', 0) for i in range(1, K_p + 1)}
        gamma_neg = {i: params.get(f'd_lp_neg_l{i}', 0) for i in range(1, K_p + 1)}
    else:  # negative
        beta = {i: params.get(f'd_lc_neg_l{i}', 0) for i in range(K_c + 1)}
        gamma_pos = {i: params.get(f'd_lp_pos_l{i}', 0) for i in range(1, K_p + 1)}
        gamma_neg = {i: params.get(f'd_lp_neg_l{i}', 0) for i in range(1, K_p + 1)}
    
    # Compute cumulative pass-through
    B = [beta[0]]
    
    for k in range(1, K_c + 1):
        # Direct effect from cost change at lag k
        term_beta = beta.get(k, 0)
        
        # Autoregressive feedback from previous price changes
        term_ar = 0
        for j in range(1, min(K_p + 1, k + 1)):
            if k - j >= 0:
                if k - j == 0:
                    prev_change = B[0]
                else:
                    prev_change = B[k - j] - B[k - j - 1]
                
                # Use appropriate gamma based on sign of previous change
                if prev_change > 0:
                    term_ar += gamma_pos.get(j, 0) * prev_change
                else:
                    term_ar += gamma_neg.get(j, 0) * prev_change
        
        # Total change in pass-through at period k
        delta_B_k = term_beta + term_ar
        B.append(B[k - 1] + delta_B_k)
    
    return np.array(B)

In [14]:
# =============================================================================
# Baseline Estimationa
# =============================================================================

K_c = 52  # Number of lags
K_p = 4
L = 0   # Number of leads (placebo)

model_asym, lead_vars, lag_vars_cost_pos, lag_vars_cost_neg, lag_vars_price_pos, lag_vars_price_neg = estimate_passthrough_asymmetric(df_weekly, K_c=K_c, K_p=K_p, L_leads = L)
print(model_asym.summary())

                            OLS Regression Results                            
Dep. Variable:                   d_lp   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     63.16
Date:                Sun, 01 Feb 2026   Prob (F-statistic):          1.10e-202
Time:                        18:00:55   Log-Likelihood:                 1223.3
No. Observations:                 521   AIC:                            -2195.
Df Residuals:                     395   BIC:                            -1658.
Df Model:                         125                                         
Covariance Type:                  HAC                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0005      0.009      0.050   

In [16]:
# -----------------------------
# 1) Settings
# -----------------------------
H = 60          # horizon to compute IRF/cumulative PT (2 years)
N_DRAWS = 3000  # parametric draws for CI
SEED = 42

# -----------------------------
# 2) Point estimates using existing function
# -----------------------------
cumulative_pt_pos = compute_cumulative_pt_asymmetric(model_asym, K_c, K_p, shock_type="positive")
cumulative_pt_neg = compute_cumulative_pt_asymmetric(model_asym, K_c, K_p, shock_type="negative")

# Extend to horizon H if needed
if len(cumulative_pt_pos) < H + 1:
    cumulative_pt_pos = np.pad(cumulative_pt_pos, (0, H + 1 - len(cumulative_pt_pos)), mode='edge')
    cumulative_pt_neg = np.pad(cumulative_pt_neg, (0, H + 1 - len(cumulative_pt_neg)), mode='edge')

# -----------------------------
# 3) Confidence intervals via parametric simulation
# -----------------------------

# Helper classes for bootstrap (defined once, outside loop)
class MockParams:
    """Mock params object that mimics statsmodels params interface."""
    def __init__(self, params_dict: dict):
        self._d = params_dict
    def get(self, key: str, default: float = 0.0) -> float:
        return self._d.get(key, default)

class MockModel:
    """Mock model object for computing cumulative PT with bootstrap parameters."""
    def __init__(self, params_dict: dict):
        self.params = MockParams(params_dict)

names = model_asym.model.exog_names
idx = {n: i for i, n in enumerate(names)}
theta_hat = model_asym.params.values
V = model_asym.cov_params().values

rng = np.random.default_rng(SEED)
draws = rng.multivariate_normal(theta_hat, V, size=N_DRAWS)

# Precompute indexes
beta_pos_idx = [idx[f"d_lc_pos_l{k}"] for k in range(K_c + 1)]
beta_neg_idx = [idx[f"d_lc_neg_l{k}"] for k in range(K_c + 1)]
gamma_pos_idx = [idx[f"d_lp_pos_l{j}"] for j in range(1, K_p + 1)]
gamma_neg_idx = [idx[f"d_lp_neg_l{j}"] for j in range(1, K_p + 1)]

cum_draws_pos = []
cum_draws_neg = []

for r in range(N_DRAWS):
    params_dict = {name: draws[r, i] for i, name in enumerate(names)}
    mock_model = MockModel(params_dict)
    
    cum_pos = compute_cumulative_pt_asymmetric(mock_model, K_c, K_p, shock_type="positive")
    cum_neg = compute_cumulative_pt_asymmetric(mock_model, K_c, K_p, shock_type="negative")
    
    # Pad if needed
    if len(cum_pos) < H + 1:
        cum_pos = np.pad(cum_pos, (0, H + 1 - len(cum_pos)), mode='edge')
        cum_neg = np.pad(cum_neg, (0, H + 1 - len(cum_neg)), mode='edge')
    
    cum_draws_pos.append(cum_pos)
    cum_draws_neg.append(cum_neg)

cum_draws_pos = np.array(cum_draws_pos)
cum_draws_neg = np.array(cum_draws_neg)

ci_lower_pos = np.percentile(cum_draws_pos, 2.5, axis=0)
ci_upper_pos = np.percentile(cum_draws_pos, 97.5, axis=0)
ci_lower_neg = np.percentile(cum_draws_neg, 2.5, axis=0)
ci_upper_neg = np.percentile(cum_draws_neg, 97.5, axis=0)

horizons = np.arange(H + 1)

# -----------------------------
# 4) Plot both curves
# -----------------------------
fig = go.Figure()

# Positive shock - CI
fig.add_trace(
    go.Scatter(
        x=np.concatenate([horizons, horizons[::-1]]),
        y=np.concatenate([ci_upper_pos, ci_lower_pos[::-1]]),
        fill="toself",
        fillcolor="rgba(0, 38, 118, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        showlegend=True,
        name="95% CI (cost increase)",
        hoverinfo="skip",
    )
)

fig.add_trace(
    go.Scatter(
        x=horizons,
        y=cumulative_pt_pos,
        mode="lines+markers",
        name="Cost increase",
        line=dict(color=BERKELEY_BLUE, width=3),
        marker=dict(size=6, color=BERKELEY_BLUE),
    )
)

# Negative shock - CI
fig.add_trace(
    go.Scatter(
        x=np.concatenate([horizons, horizons[::-1]]),
        y=np.concatenate([ci_upper_neg, ci_lower_neg[::-1]]),
        fill="toself",
        fillcolor="rgba(253, 181, 21, 0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        showlegend=True,
        name="95% CI (cost decrease)",
        hoverinfo="skip",
    )
)

fig.add_trace(
    go.Scatter(
        x=horizons,
        y=cumulative_pt_neg,
        mode="lines+markers",
        name="Cost decrease",
        line=dict(color=CALIFORNIA_GOLD, width=3),
        marker=dict(size=6, color=CALIFORNIA_GOLD),
    )
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", line_width=1.5)
fig.add_hline(y=1, line_dash="dot", line_color="red", line_width=1.5)
fig.add_hline(y=-1, line_dash="dot", line_color="red", line_width=1.5)

fig.update_layout(
    title="Asymmetric Cumulative Pass-Through",
    xaxis_title="Weeks after cost shock (h)",
    yaxis_title="Cumulative pass-through",
    font=dict(family="Times New Roman", size=14, color="black"),
    template="simple_white",
    xaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    yaxis=dict(
        showgrid=True, gridcolor="lightgray", zeroline=False,
        linecolor="black", linewidth=1, mirror=True,
        ticks="outside", tickwidth=1, tickcolor="black",
    ),
    legend=dict(
        x=1.02, y=1, xanchor="left", yanchor="top",
        font=dict(size=15), bgcolor="rgba(255,255,255,0)", bordercolor="rgba(0,0,0,0)",
    ),
    margin=dict(r=150),
    width=850,
    height=520,
)

fig.show()
fig.write_image(PLOT_FOLDER / "Cumulative_PassThrough_Asymmetric.png")

# -----------------------------
# 5) Print key horizons
# -----------------------------
print("\nCumulative Pass-Through at Key Horizons:")
print("=" * 80)
for h in [4, 13, 26, 52]:
    if h <= H:
        print(f"\nWeek {h}:")
        print(f"  Cost increase:  {cumulative_pt_pos[h]:6.3f}  [{ci_lower_pos[h]:6.3f}, {ci_upper_pos[h]:6.3f}]")
        print(f"  Cost decrease:  {cumulative_pt_neg[h]:6.3f}  [{ci_lower_neg[h]:6.3f}, {ci_upper_neg[h]:6.3f}]")

In [17]:
con.close()